In [2]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import os
from tqdm import tqdm
pd.set_option('display.max_colwidth', None) 

In [7]:
frame_path = pd.DataFrame()
for root, dirs, files in os.walk('./frames/frame error/AM-FM CP AM-FM'):
    for name in files:
        index = name[5:-11]
        frame_path.loc[int(index),'path'] = os.path.join(root,name)

In [8]:
frame_path = frame_path.sort_index()
frame_path

,path
0,./frames/frame error/AM-FM CP AM-FM/frame0_000000.jpg
1,./frames/frame error/AM-FM CP AM-FM/frame1_000000.jpg
2,./frames/frame error/AM-FM CP AM-FM/frame2_000000.jpg
3,./frames/frame error/AM-FM CP AM-FM/frame3_000000.jpg
4,./frames/frame error/AM-FM CP AM-FM/frame4_000000.jpg
...,...
1571,./frames/frame error/AM-FM CP AM-FM/frame1571_000105.jpg
1572,./frames/frame error/AM-FM CP AM-FM/frame1572_000105.jpg
1573,./frames/frame error/AM-FM CP AM-FM/frame1573_000105.jpg
1574,./frames/frame error/AM-FM CP AM-FM/frame1574_000105.jpg


In [9]:
diff = []
for i in tqdm(range(len(frame_path[:-1]))):
    diff.append(np.mean(cv2.imread(frame_path['path'].iloc[i+1],0).astype(np.int16)-cv2.imread(frame_path['path'].iloc[i],0).astype(np.int16)))

100%|█████████████████████████████████████████████████████████████████████████████████████████| 1575/1575 [00:22<00:00, 70.28it/s]


In [10]:
sum_list = []
threshold_list = []
for window in range(3,8):
    sum_list_for_each_window = []
    threshold_list_for_each_window = []
    for i in range(len(diff[:-window])):
        temp_sum = 0
        temp_abs_sum = 0
        for value in diff[i:i+window]:
            if abs(value) > 1: # only significant frame
                temp_sum += value
                temp_abs_sum += abs(value)
        sum_list_for_each_window.append(temp_sum)
        threshold_list_for_each_window.append(temp_abs_sum)
    sum_list.append(sum_list_for_each_window)
    threshold_list.append(threshold_list_for_each_window)

In [11]:
flicker_detected = pd.DataFrame(columns=['start_frame','window'])
for window in range(3,8):
    for i in range(len(diff[:-window])):
        if (diff[i] > 4 or diff[i] < -4) and abs(sum_list[window - 3][i]) < max(0.5, threshold_list[window - 3][i] / 40) and i not in flicker_detected["start_frame"].values:
            flicker_detected.loc[len(flicker_detected)] = [i, window]           

flicker_detected = flicker_detected.sort_values(by='start_frame')
for i in range(len(flicker_detected)):
    frame, window = flicker_detected.iloc[i].values
    timestamp = frame_path['path'].iloc[frame]
    print(f'{frame}\t{timestamp[-10:-8]}:{timestamp[-8:-6]}:{timestamp[-6:-4]}\twindow={window}')

# max(0.5, threshold_list[window - 3][i] / 40)

23	00:00:00	window=5
453	00:00:18	window=4
809	00:00:33	window=4
1137	00:00:47	window=4
1378	00:00:57	window=3


In [152]:
diff[730:735]

[4.47791015625,
 -0.5528797743055556,
 -0.6020399305555556,
 -1.8600694444444446,
 -2.1453960503472222]

In [43]:
# sum_list = []
# window = 5

# for i in range(len(diff[:-window])):
#     temp_sum = 0
#     for value in diff[i:i+window]:
#         if abs(value) > 1.2: # only significant frame
#             temp_sum += value
#     sum_list.append(temp_sum)

In [44]:
# flicker_detected = []
# for i in range(len(diff[:-window])):
#     if (diff[i] > 4 or diff[i] < -4) and abs(sum_list[i]) < 0.5:
#         flicker_detected.append(i)           

# for frame in sorted(flicker_detected):
#     timestamp = frame_path['path'].iloc[frame]
#     print(f'{frame}\t{timestamp[-10:-8]}:{timestamp[-8:-6]}:{timestamp[-6:-4]}')

162	00:00:16


In [54]:
sum_list[0][65:70]

[13.034951171875,
 14.038486328125,
 0.05139431423611107,
 0.05139431423611107,
 16.605608723958333]